<a href="https://colab.research.google.com/github/shyamjith94/Machine-Learning-Regression/blob/main/FeatureSelectionAndLASSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%%capture
!pip install turicreate

In [21]:
import turicreate as tc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from math import log, sqrt
from sklearn.utils import shuffle

In [22]:
data = tc.SFrame.read_csv("/content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv")

Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.140288 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/drive/My Drive/Machine Learning: Regression/Data/2/c_kc_house_data.csv/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.104096 secs.

In [23]:
data['sqft_living_sqrt'] = data['sqft_living'].apply(sqrt)
data['sqft_lot_sqrt'] = data['sqft_lot'].apply(sqrt)
data['bedrooms_square'] = data['bedrooms']*data['bedrooms']

data['floors'] = data['floors'].astype(float) 
data['floors_square'] = data['floors']*data['floors']

In [24]:
data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
7129300520,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0
5631500400,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0
2487200875,20141209T000000,604000.0,4,3.0,1960,5000,1.0,0,0
1954400510,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0
7237550310,20140512T000000,1225000.0,4,4.5,5420,101930,1.0,0,0
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0
2008000270,20150115T000000,291850.0,3,1.5,1060,9711,1.0,0,0
2414600126,20150415T000000,229500.0,3,1.0,1780,7470,1.0,0,0
3793500160,20150312T000000,323000.0,3,2.5,1890,6560,2.0,0,0


In [25]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

**Linear regression model with L1 penalty**

In [26]:
model_all = tc.linear_regression.create(data, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=1e10)

**Explore weights in the model**

In [27]:
model_all_weights = model_all._get('coefficients')
model_all_weights.sort('value', ascending=False)

name,index,value,stderr
(intercept),None,274873.05565166846,None
bathrooms,None,8468.531052082433,None
grade,None,842.0680288184716,None
sqft_living_sqrt,None,350.06055100460185,None
sqft_living,None,24.4207211037497,None
sqft_above,None,20.024722643562534,None
floors_square,None,0.0,None
yr_renovated,None,0.0,None
yr_built,None,0.0,None
sqft_basement,None,0.0,None


In [28]:
(training_and_validation, testing) = data.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

In [29]:
def cross_validation(l1_penalty, train, validation, output_name, feature_list):
    """ l1_penalty  :- l1 penalty value
        train       :- train set
        validation  :- validation set
        output_name :- output label name
        feature_list:- feature list train label
        return      :- validation error
        Using oly l1 penalty keep l2 penalty zero
        create model predict value check the validation data set error
        """
    model = tc.linear_regression.create(train, target = output_name, 
                                            features = feature_list, 
                                            verbose = False,
                                            l1_penalty = l1_penalty,
                                            l2_penalty=0. ,
                                            validation_set = None)
    validation_prediction = model.predict(validation)
    validation_error = ((validation_prediction - validation[output_name])**2).sum()
    return validation_error


In [30]:
l1_penalty_validation_error = []
l1_penalty_values = []

for l1_penalty in np.linspace(1, 7, 13):
    l1_penalty_values.append(l1_penalty)
    validation_error = cross_validation(l1_penalty, training, validation, 'price', all_features)
    l1_penalty_validation_error.append(validation_error)

**The best value for the l1_penalty**

In [31]:
for l1, error in zip(l1_penalty_values, l1_penalty_validation_error):
    if error == min(l1_penalty_validation_error):
        print(l1)



1.0


In [32]:

model_best_l1 = tc.linear_regression.create(data, target='price', features=all_features,
                                              validation_set=None, verbose= False,
                                              l2_penalty=0., l1_penalty=1.0)

In [33]:
model_best_l1 = model_best_l1._get('coefficients').sort('value')
model_best_l1

name,index,value,stderr
sqft_lot,None,-0.016849913266912184,None
yr_built,None,10.188166826770491,None
sqft_living,None,37.49115059069754,None
sqft_above,None,41.349739271655004,None
yr_renovated,None,58.711583781715504,None
sqft_basement,None,118.2324208192088,None
sqft_lot_sqrt,None,149.5694252428369,None
sqft_living_sqrt,None,1109.3959681838858,None
bedrooms_square,None,1479.7378812106285,None
grade,None,6205.641050927333,None


In [35]:
model_best_l1['value'].nnz()

18

In [45]:
model_best_l1 = tc.linear_regression.create(data, target='price', features=all_features,
                                              validation_set=None, verbose= False,
                                              l2_penalty=0., l1_penalty=18)

In [46]:
model_best_l1 = model_best_l1._get('coefficients').sort('value')
model_best_l1

name,index,value,stderr
sqft_lot,None,-0.016849916585686638,None
yr_built,None,10.18816679724043,None
sqft_living,None,37.49115058555033,None
sqft_above,None,41.349739253133336,None
yr_renovated,None,58.71158320684832,None
sqft_basement,None,118.23242056072891,None
sqft_lot_sqrt,None,149.56942492885017,None
sqft_living_sqrt,None,1109.3959676371219,None
bedrooms_square,None,1479.7378752068132,None
grade,None,6205.641045794437,None


In [47]:
model_best_l1['value'].nnz()

18

In [48]:
max_nonzeros = 7

In [49]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([1.00000000e+08, 1.27427499e+08, 1.62377674e+08, 2.06913808e+08,
       2.63665090e+08, 3.35981829e+08, 4.28133240e+08, 5.45559478e+08,
       6.95192796e+08, 8.85866790e+08, 1.12883789e+09, 1.43844989e+09,
       1.83298071e+09, 2.33572147e+09, 2.97635144e+09, 3.79269019e+09,
       4.83293024e+09, 6.15848211e+09, 7.84759970e+09, 1.00000000e+10])

In [50]:
l1_penalty_validation_error = []
non_zero_parameters = []

In [54]:
for l1_penalty in l1_penalty_values:
    model = tc.linear_regression.create(training, features=all_features, target='price', verbose=False,
                                        l2_penalty=0, l1_penalty=l1_penalty, validation_set=None)
    validation_predicted = model.predict(validation)
    validation_error = ((validation_predicted - validation['price'])**2).sum()
    model_coff = model._get('coefficients').sort('value')
    model_non_zero_weight = model_coff['value'].nnz()

    l1_penalty_validation_error.append(validation_error)
    non_zero_parameters.append(model_non_zero_weight)



In [60]:
for l1, weight in zip(l1_penalty_values, non_zero_parameters):
    print("l1_penalty:\t","{:.4f}".format(l1), "\tweight:-\t",weight)

l1_penalty:	 100000000.0000 	weight:-	 18
l1_penalty:	 127427498.5703 	weight:-	 18
l1_penalty:	 162377673.9189 	weight:-	 18
l1_penalty:	 206913808.1115 	weight:-	 18
l1_penalty:	 263665089.8730 	weight:-	 17
l1_penalty:	 335981828.6284 	weight:-	 17
l1_penalty:	 428133239.8719 	weight:-	 17
l1_penalty:	 545559478.1169 	weight:-	 17
l1_penalty:	 695192796.1776 	weight:-	 17
l1_penalty:	 885866790.4101 	weight:-	 16
l1_penalty:	 1128837891.6847 	weight:-	 15
l1_penalty:	 1438449888.2877 	weight:-	 15
l1_penalty:	 1832980710.8324 	weight:-	 13
l1_penalty:	 2335721469.0901 	weight:-	 12
l1_penalty:	 2976351441.6313 	weight:-	 10
l1_penalty:	 3792690190.7323 	weight:-	 6
l1_penalty:	 4832930238.5718 	weight:-	 5
l1_penalty:	 6158482110.6603 	weight:-	 3
l1_penalty:	 7847599703.5146 	weight:-	 1
l1_penalty:	 10000000000.0000 	weight:-	 1
